In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import spawn_camera, spawn_vehicle, preprocess_image, spawn_camera_depth, riconosci_parcheggio1, follow_curve2, control_retro

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
client.load_world('Town05')
world = client.get_world()
spectator = world.get_spectator()

In [8]:
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
spawn_point = carla.Transform(carla.Location(-2, -25, 2), carla.Rotation(yaw=-90))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
camera_retro = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-1.6, z=1.9), carla.Rotation(yaw=180, pitch=-40)))
camera_rigth = spawn_camera(attach_to=vehicle, world=world, transform=carla.Transform(carla.Location(x=-0.8, y=0.5, z=1.9), carla.Rotation(yaw=90, pitch=-40)))
run = True

In [9]:
def camera_callback(image):
    global run
    prediction=riconosci_parcheggio1(preprocess_image(np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))))
    #print(prediction)
    #cv2.imwrite(f'./output5/{image.frame}.png', np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)))
    if prediction > 0.6:
        run = False
camera_rigth.listen(lambda image: camera_callback(image))

In [10]:
def controllo_velocita(control, target_speed_mps, current_speed_mps):
    speed_error = target_speed_mps - current_speed_mps
    if speed_error > 0:
        control.throttle = min(1.0, 0.5 + speed_error * 0.5) 
        control.brake = 0
    elif speed_error < 0:
        control.throttle = 0.0
        control.brake = min(1.0, -speed_error * 0.5) 
    else:
        control.throttle = 0.0
        control.brake = 0.0
    return control

In [11]:
def bezier_quadratica(p0, p1, p2, t_values=np.linspace(0, 1, 30)):
    p0 = np.array(p0)  # Punto iniziale
    p1 = np.array(p1)  # Punto di controllo
    p2 = np.array(p2)  # Punto finale
    points = np.array([(1 - t)**2 * p0 + 2 * (1 - t) * t * p1 + t**2 * p2 for t in t_values])
    return points

def find_highest_segment_midpoint_and_perpendicular(mask):
    """
    Trova il segmento più alto in un'immagine binaria utilizzando la trasformata di Hough, calcola
    il punto medio e determina una retta perpendicolare al segmento.

    Args:
        mask (numpy.ndarray): Immagine binaria (0 e 255).

    Returns:
        tuple: Coordinata (x, y) del punto medio e punti di inizio e fine della retta perpendicolare.
    """
    # Trova linee con la trasformata di Hough
    lines = cv2.HoughLinesP(mask, rho=1, theta=np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

    if lines is None:
        return None, None, None  # Nessuna linea trovata

    # Trova il segmento più alto
    highest_segment = None
    min_y = float('inf')

    for line in lines:
        x1, y1, x2, y2 = line[0]
        avg_y = (y1 + y2) / 2
        if avg_y < min_y:
            min_y = avg_y
            highest_segment = (x1, y1, x2, y2)

    if highest_segment is None:
        return None, None, None  # Nessun segmento valido trovato

    x1, y1, x2, y2 = highest_segment

    # Calcola il punto medio del segmento
    midpoint = ((x1 + x2) // 2, (y1 + y2) // 2)

    # Calcola la pendenza del segmento
    if x2 != x1:  # Evita divisioni per zero
        slope = (y2 - y1) / (x2 - x1)
    else:
        slope = float('inf')  # Segmento verticale

    # Calcola la pendenza della retta perpendicolare
    if slope != 0 and slope != float('inf'):
        perp_slope = -1 / slope
    else:
        perp_slope = 0 if slope == float('inf') else float('inf')

    # Calcola i punti di inizio e fine della retta perpendicolare
    length = 50  # Lunghezza della retta perpendicolare (metà sopra e metà sotto il punto medio)
    if perp_slope == float('inf'):
        perp_start = (midpoint[0], midpoint[1] - length)
        perp_end = (midpoint[0], midpoint[1] + length)
    elif perp_slope == 0:
        perp_start = (midpoint[0] - length, midpoint[1])
        perp_end = (midpoint[0] + length, midpoint[1])
    else:
        dx = int(length / math.sqrt(1 + perp_slope**2))
        dy = int(perp_slope * dx)
        perp_start = (midpoint[0] - dx, midpoint[1] - dy)
        perp_end = (midpoint[0] + dx, midpoint[1] + dy)

    return midpoint, perp_start, perp_end

def curva(img):
    img = preprocess_image(img)
    output_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)  # Converti in BGR per colori
    cv2.line(output_img, (0, 450), (800, 350), (255, 255, 255), thickness=8) 
    img = cv2.cvtColor(output_img, cv2.COLOR_BGR2GRAY)
    colored_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    contours, _ = cv2.findContours(255 - img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    height, width = img.shape
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        if x > 0 and y > 0 and (x + w) < width and (y + h) < height:
            if area > 1000: 
                print(area)
                cv2.drawContours(colored_image, [contour], -1, (0,0,255), thickness=cv2.FILLED)

    hsv_image = cv2.cvtColor(colored_image, cv2.COLOR_BGR2HSV)

    # Definisci i range per il colore rosso in formato HSV
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])

    # Maschera per isolare il colore rosso
    mask = cv2.inRange(hsv_image, lower_red, upper_red)
    mask = cv2.Canny(mask,100,200)
    midpoint, perp_start, perp_end = find_highest_segment_midpoint_and_perpendicular(mask)
    #print(midpoint)
    #cv2.circle(colored_image, midpoint, 5, (255, 0, 0), -1)
    #t = np.linspace(0, 1, 20)
    #curve_points = bezier_quadratica(([400,600]), ([400, int((600 - midpoint[1]) / 2)]), ([midpoint[0],midpoint[1]]))
    return midpoint

In [12]:
target_speed_mps = 10 / 3.6

control = carla.VehicleControl()
control.steer = 0.0
control.brake = 0.0
target_distance = 1.1
distance_travelled = 0.0

while run:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.1)
while distance_travelled < target_distance:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * 0.1
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
control.brake = 0.0
control.reverse = True
control.steer = 0.8
distance_travelled = 0.0
target_distance = 3.5
while distance_travelled < target_distance:
    current_velocity = vehicle.get_velocity()
    current_speed_mps = current_velocity.length()
    distance_travelled += current_speed_mps * 0.1
    print(control)
    control = controllo_velocita(control, target_speed_mps, current_speed_mps)
    vehicle.apply_control(control)
    time.sleep(0.1)
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
camera_rigth.destroy()
image1 = None
def camera_callback2(image):
    global image1
    image1 =  np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
camera_retro.listen(lambda image: camera_callback2(image))
time.sleep(0.1)
#follow_curve2(world, vehicle, curva(image1), control=control, target_speed_mps=10/3.6)
while True:
    if control_retro(vehicle, curva(image1), control):
        break
control.brake = 1.0
control.throttle = 0.0
vehicle.apply_control(control)
time.sleep(4.1)
camera_retro.destroy()
vehicle.destroy()

VehicleControl(throttle=0.000000, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.534487, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=1.000000, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=1.000000, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=1.000000, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=1.000000, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=1.000000, steer=0.800000, brake=0.000000, hand_brake=False, reverse=True, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.918578, steer=0.800000, brake=0.000000, hand_brake=False,

True